In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

import gc
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
class ANN(nn.Module):
    def __init__(
        self,
        in_dim: int,
        hidden_dim_1: int,
        hidden_dim_2: int,
        hidden_dim_3: int,
        n_classes:int = 10,
        dropout: float = 0.3
    ):
        super().__init__()
        
        self.layer1 = nn.Sequential(
            nn.Linear(in_features=in_dim, out_features=hidden_dim_1),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim_1),
            nn.Dropout(dropout),
        )
        self.layer2 = nn.Sequential(
            nn.Linear(in_features=hidden_dim_1, out_features=hidden_dim_2),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim_2),
            nn.Dropout(dropout),
        )
        self.layer3 = nn.Sequential(
            nn.Linear(in_features=hidden_dim_2, out_features=hidden_dim_3),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim_3),
            nn.Dropout(dropout),
        )
        self.output_layer = nn.Linear(in_features=hidden_dim_3, out_features=n_classes)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
            Args:
                x (torch.Tensor): (batch_size, in_dim) the input
            
            Output:
                (torch.Tensor): (batch_size, n_classes) the output
        """
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.output_layer(x)
        
        return x

In [4]:
class MNIST(Dataset):
    def __init__(
        self,
        data,
    ):
        self.data = data
    # def _build(self):
        # scaler = MinMaxScaler(feature_range=())
        # scaler = StandardScaler()
        
    def __getitem__(self, index) -> (torch.Tensor, torch.Tensor):
        return torch.tensor(self.data.iloc[index, 1:], dtype=torch.float32).to(device), torch.tensor(self.data.iloc[index, 0]).to(device)
    
    def __len__(self):
        return self.data.shape[0]

In [6]:
train = pd.read_csv('/kaggle/input/mnist-in-csv/mnist_train.csv')
test = pd.read_csv('/kaggle/input/mnist-in-csv/mnist_test.csv')

In [7]:
scaler = StandardScaler()
train.iloc[:, 1:] = scaler.fit_transform(X=train.iloc[:, 1:])
test.iloc[:, 1:] = scaler.transform(X=test.iloc[:, 1:])

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasatt

In [8]:
train_dataset = MNIST(data=train)
test_dataset = MNIST(data=test)

In [9]:
train_batchsize = 512
val_batchsize = 512

In [10]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=train_batchsize, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=val_batchsize, shuffle=True)

In [11]:
model = ANN(
    in_dim=784,
    hidden_dim_1=784//2,
    hidden_dim_2=784//4,
    hidden_dim_3=784//8
).to(device)

In [12]:
n_epochs = 20

In [13]:
lr = 1e-3
optimiser = torch.optim.Adam(model.parameters(), lr=lr)

loss_fn = torch.nn.CrossEntropyLoss()

In [14]:
def train_epoch(
    model,
    dataloader,
    optimiser
):
    model.train()
    
    for batch in tqdm(dataloader):
        x, y = batch[0], batch[1]
        
        output = model(x)
        output = nn.Softmax(dim=-1)(output)
        loss = loss_fn(output, y)
        
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
        
        if sanity_check:
            break
        
def validate(
    model,
    dataloader
):
    model.eval()
    total_loss = 0
    predictions = []
    truths = []
    
    with torch.no_grad():
        for batch in tqdm(dataloader):
            x, y = batch[0], batch[1]
            
            output = model(x)
            output = nn.Softmax(dim=-1)(output)
            loss = loss_fn(output, y)
            total_loss += loss.detach().cpu().item()/len(dataloader)
            
            preds = torch.argmax(output, dim=-1)
            predictions.extend(preds.cpu())
            truths.extend(y.cpu())
            
            if sanity_check:
                break
        
    acc = accuracy_score(y_true=truths, y_pred=predictions)
    f1 = f1_score(y_true=truths, y_pred=predictions, average='macro')
    
    return total_loss, acc, f1

In [15]:
def train_model(
    model,
    train_dataloader,
    test_dataloader,
    optimiser,
):
    for epoch in range(1, n_epochs+1):
        start_time = time.time()
        
        print(f"========= EPOCH {epoch} STARTED =========")
        train_epoch(model=model, dataloader=train_dataloader, optimiser=optimiser)
        
        print(f"========= TRAIN EVALUATION STARTED =========")
        train_val_op = validate(model=model, dataloader=train_dataloader)
        
        print(f"========= TEST EVALUATION STARTED =========")
        test_val_op = validate(model=model, dataloader=test_dataloader)
        
        print(f"END OF {epoch} EPOCH")
        print(f"| Time taken: {time.time() - start_time: 7.3f} |")
        print(f"| Train Loss: {train_val_op[0]: 7.3f} | Train acc: {train_val_op[1]: 1.5f} | Train f1: {train_val_op[2]: 1.5f} |")
        print(f"| Test Loss: {test_val_op[0]: 7.3f}  | Test acc: {test_val_op[1]: 1.5f}  | Test f1: {test_val_op[2]: 1.5f}  |")
        
        if sanity_check:
            break
        

In [16]:
sanity_check=False

In [17]:
train_model(
    model=model,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    optimiser=optimiser,
)

========= EPOCH 1 STARTED =========


  0%|          | 0/118 [00:00<?, ?it/s]/tmp/ipykernel_47/2879017467.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return torch.tensor(self.data.iloc[index, 1:], dtype=torch.float32).to(device), torch.tensor(self.data.iloc[index, 0]).to(device)
100%|██████████| 118/118 [02:14<00:00,  1.14s/it]


========= TRAIN EVALUATION STARTED =========


100%|██████████| 118/118 [02:11<00:00,  1.12s/it]


========= TEST EVALUATION STARTED =========


...

========= TRAIN EVALUATION STARTED =========


100%|██████████| 118/118 [02:11<00:00,  1.12s/it]


========= TEST EVALUATION STARTED =========


  0%|          | 0/20 [00:00<?, ?it/s]/tmp/ipykernel_47/2879017467.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return torch.tensor(self.data.iloc[index, 1:], dtype=torch.float32).to(device), torch.tensor(self.data.iloc[index, 0]).to(device)
100%|██████████| 20/20 [00:21<00:00,  1.09s/it]


END OF 20 EPOCH
| Time taken:  292.908 |
| Train Loss:   1.469 | Train acc:  0.99247 | Train f1:  0.99247 |
| Test Loss:   1.484  | Test acc:  0.97730  | Test f1:  0.97716  |
